
# Sample Data Warehouse Builder
This notebook seeds `Employee_Information.db` with 10 relational tables representing a light-weight commerce and support data model. Each table contains between 10 and 50 synthetic rows so the dataset is useful for experimenting with SQL joins and BI/analytics exercises.


In [1]:

import sqlite3
import random
from datetime import datetime, timedelta

DB_PATH = "Employee_Information.db"
random.seed(42)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.executescript(
    """
    PRAGMA foreign_keys = OFF;
    DROP TABLE IF EXISTS order_items;
    DROP TABLE IF EXISTS payments;
    DROP TABLE IF EXISTS support_tickets;
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS addresses;
    DROP TABLE IF EXISTS user_profiles;
    DROP TABLE IF EXISTS users;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS states;
    DROP TABLE IF EXISTS countries;
    DROP TABLE IF EXISTS employee_records;
    DROP TABLE IF EXISTS departments;
    PRAGMA foreign_keys = ON;
    """
)

TABLE_DEFINITIONS = [
    (
        "countries",
        """CREATE TABLE IF NOT EXISTS countries (
            country_id INTEGER PRIMARY KEY,
            country_name TEXT NOT NULL,
            iso_code TEXT NOT NULL UNIQUE,
            region TEXT,
            population INTEGER
        )""",
    ),
    (
        "states",
        """CREATE TABLE IF NOT EXISTS states (
            state_id INTEGER PRIMARY KEY,
            state_name TEXT NOT NULL,
            country_id INTEGER NOT NULL,
            abbreviation TEXT,
            state_capital TEXT,
            FOREIGN KEY (country_id) REFERENCES countries(country_id)
        )""",
    ),
    (
        "users",
        """CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            country_id INTEGER,
            signup_date TEXT,
            FOREIGN KEY (country_id) REFERENCES countries(country_id)
        )""",
    ),
    (
        "user_profiles",
        """CREATE TABLE IF NOT EXISTS user_profiles (
            profile_id INTEGER PRIMARY KEY,
            user_id INTEGER UNIQUE,
            birth_date TEXT,
            gender TEXT,
            language_preference TEXT,
            loyalty_tier TEXT,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )""",
    ),
    (
        "addresses",
        """CREATE TABLE IF NOT EXISTS addresses (
            address_id INTEGER PRIMARY KEY,
            user_id INTEGER,
            address_line1 TEXT,
            city TEXT,
            state TEXT,
            postal_code TEXT,
            country_id INTEGER,
            FOREIGN KEY (user_id) REFERENCES users(user_id),
            FOREIGN KEY (country_id) REFERENCES countries(country_id)
        )""",
    ),
    (
        "products",
        """CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL,
            category TEXT,
            unit_price REAL,
            availability_status TEXT
        )""",
    ),
    (
        "orders",
        """CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY,
            user_id INTEGER,
            order_date TEXT,
            total_amount REAL,
            currency TEXT,
            status TEXT,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )""",
    ),
    (
        "order_items",
        """CREATE TABLE IF NOT EXISTS order_items (
            order_item_id INTEGER PRIMARY KEY,
            order_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            line_total REAL,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )""",
    ),
    (
        "payments",
        """CREATE TABLE IF NOT EXISTS payments (
            payment_id INTEGER PRIMARY KEY,
            order_id INTEGER,
            payment_method TEXT,
            payment_date TEXT,
            payment_amount REAL,
            payment_status TEXT,
            FOREIGN KEY (order_id) REFERENCES orders(order_id)
        )""",
    ),
    (
        "support_tickets",
        """CREATE TABLE IF NOT EXISTS support_tickets (
            ticket_id INTEGER PRIMARY KEY,
            user_id INTEGER,
            issue_type TEXT,
            priority TEXT,
            status TEXT,
            created_at TEXT,
            resolved_at TEXT,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )""",
    ),
]

for _, ddl in TABLE_DEFINITIONS:
    cursor.execute(ddl)

countries_data = [
    (1, "United States", "US", "North America", 331002651),
    (2, "Canada", "CA", "North America", 37742154),
    (3, "Mexico", "MX", "North America", 128932753),
    (4, "Brazil", "BR", "South America", 212559417),
    (5, "United Kingdom", "GB", "Europe", 67886011),
    (6, "Germany", "DE", "Europe", 83783942),
    (7, "France", "FR", "Europe", 65273511),
    (8, "India", "IN", "Asia", 1380004385),
    (9, "China", "CN", "Asia", 1444216107),
    (10, "Australia", "AU", "Oceania", 25499884),
    (11, "Japan", "JP", "Asia", 126476461),
    (12, "South Africa", "ZA", "Africa", 59308690),
    (13, "Spain", "ES", "Europe", 46754778),
    (14, "Italy", "IT", "Europe", 60461826),
    (15, "Singapore", "SG", "Asia", 5850342),
]

cursor.executemany(
    "INSERT INTO countries (country_id, country_name, iso_code, region, population) VALUES (?, ?, ?, ?, ?)",
    countries_data,
)

states_data = [
    (1, "California", 1, "CA", "Sacramento"),
    (2, "Texas", 1, "TX", "Austin"),
    (3, "Ontario", 2, "ON", "Toronto"),
    (4, "British Columbia", 2, "BC", "Victoria"),
    (5, "Jalisco", 3, "JA", "Guadalajara"),
    (6, "Mexico City", 3, "MX", "Mexico City"),
    (7, "Sao Paulo", 4, "SP", "Sao Paulo"),
    (8, "Rio de Janeiro", 4, "RJ", "Rio de Janeiro"),
    (9, "England", 5, "ENG", "London"),
    (10, "Scotland", 5, "SCO", "Edinburgh"),
    (11, "Bavaria", 6, "BY", "Munich"),
    (12, "Hesse", 6, "HE", "Wiesbaden"),
    (13, "Ile-de-France", 7, "IDF", "Paris"),
    (14, "Occitanie", 7, "OCC", "Toulouse"),
    (15, "Maharashtra", 8, "MH", "Mumbai"),
    (16, "Karnataka", 8, "KA", "Bengaluru"),
    (17, "Guangdong", 9, "GD", "Guangzhou"),
    (18, "Sichuan", 9, "SC", "Chengdu"),
    (19, "New South Wales", 10, "NSW", "Sydney"),
    (20, "Victoria", 10, "VIC", "Melbourne"),
]

cursor.executemany(
    "INSERT INTO states (state_id, state_name, country_id, abbreviation, state_capital) VALUES (?, ?, ?, ?, ?)",
    states_data,
)

first_names = [
    "Olivia", "Liam", "Emma", "Noah", "Ava", "Isabella", "Sophia", "Mia", "Charlotte", "Amelia",
    "Ethan", "Mason", "Logan", "Lucas", "Harper", "Evelyn", "Abigail", "Emily", "Scarlett", "Madison",
]

last_names = [
    "Brown", "Smith", "Johnson", "Williams", "Jones", "Garcia", "Miller", "Davis", "Martinez", "Hernandez",
    "Lopez", "Gonzalez", "Wilson", "Anderson", "Thomas", "Taylor", "Moore", "Jackson", "Martin", "Lee",
]

domains = ["example.com", "sample.io", "demo.org"]
country_ids = [row[0] for row in countries_data]

users_data = []
base_signup = datetime(2024, 6, 1)
for user_id in range(1, 31):
    first = random.choice(first_names)
    last = random.choice(last_names)
    email = f"{first.lower()}.{last.lower()}{user_id}@{random.choice(domains)}"
    country_id = random.choice(country_ids)
    signup_date = (base_signup - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")
    users_data.append((user_id, first, last, email, country_id, signup_date))

cursor.executemany(
    "INSERT INTO users (user_id, first_name, last_name, email, country_id, signup_date) VALUES (?, ?, ?, ?, ?, ?)",
    users_data,
)

genders = ["Female", "Male", "Non-binary"]
languages = ["English", "Spanish", "French", "German", "Hindi", "Mandarin"]
loyalty_tiers = ["Bronze", "Silver", "Gold", "Platinum"]

profiles_data = []
for user_id, *_ in users_data:
    birth_year = random.randint(1978, 2003)
    birth_date = datetime(birth_year, random.randint(1, 12), random.randint(1, 28))
    profiles_data.append(
        (
            user_id,
            user_id,
            birth_date.strftime("%Y-%m-%d"),
            random.choice(genders),
            random.choice(languages),
            random.choice(loyalty_tiers),
        )
    )

cursor.executemany(
    "INSERT INTO user_profiles (profile_id, user_id, birth_date, gender, language_preference, loyalty_tier) VALUES (?, ?, ?, ?, ?, ?)",
    profiles_data,
)

street_names = ["Oak", "Maple", "Pine", "Cedar", "Elm", "Birch", "Walnut", "Willow"]
cities = [
    "Seattle", "Toronto", "Berlin", "Paris", "Mumbai", "Sydney", "Chicago", "Denver", "Austin", "Vancouver",
    "Madrid", "Rome", "Cape Town", "Melbourne", "Singapore",
]

addresses_data = []
for idx, (user_id, *_rest) in enumerate(users_data, start=1):
    street_num = random.randint(101, 999)
    address_line1 = f"{street_num} {random.choice(street_names)} St"
    city = random.choice(cities)
    state = random.choice(states_data)[1]
    postal_code = f"{10000 + idx:05d}"
    country_id = random.choice(country_ids)
    addresses_data.append((idx, user_id, address_line1, city, state, postal_code, country_id))

cursor.executemany(
    "INSERT INTO addresses (address_id, user_id, address_line1, city, state, postal_code, country_id) VALUES (?, ?, ?, ?, ?, ?, ?)",
    addresses_data,
)

products_data = [
    (1, "Laptop Stand", "Accessories", 49.99, "In Stock"),
    (2, "Wireless Mouse", "Electronics", 29.5, "In Stock"),
    (3, "Mechanical Keyboard", "Electronics", 119.0, "Backordered"),
    (4, "4K Monitor", "Electronics", 329.99, "In Stock"),
    (5, "Noise Cancelling Headphones", "Audio", 249.99, "In Stock"),
    (6, "Webcam", "Electronics", 89.99, "In Stock"),
    (7, "Portable SSD", "Storage", 139.99, "Low Stock"),
    (8, "Smartphone Gimbal", "Accessories", 159.99, "In Stock"),
    (9, "Conference Speaker", "Audio", 99.5, "In Stock"),
    (10, "Desk Lamp", "Office", 39.99, "In Stock"),
    (11, "Ergonomic Chair", "Office", 449.0, "Limited"),
    (12, "Travel Adapter", "Accessories", 24.99, "In Stock"),
]

cursor.executemany(
    "INSERT INTO products (product_id, product_name, category, unit_price, availability_status) VALUES (?, ?, ?, ?, ?)",
    products_data,
)

orders_data = []
order_statuses = ["Processing", "Completed", "Shipped", "Cancelled"]
currencies = ["USD", "CAD", "EUR", "GBP", "INR", "AUD"]
for order_id in range(1, 36):
    user_id = random.choice(users_data)[0]
    order_date = (base_signup - timedelta(days=random.randint(0, 180))).strftime("%Y-%m-%d")
    total_amount = round(random.uniform(45, 620), 2)
    orders_data.append((order_id, user_id, order_date, total_amount, random.choice(currencies), random.choice(order_statuses)))

cursor.executemany(
    "INSERT INTO orders (order_id, user_id, order_date, total_amount, currency, status) VALUES (?, ?, ?, ?, ?, ?)",
    orders_data,
)

order_items_data = []
for order in orders_data:
    product = random.choice(products_data)
    quantity = random.randint(1, 4)
    line_total = round(quantity * product[3], 2)
    order_items_data.append((len(order_items_data) + 1, order[0], product[0], quantity, line_total))

cursor.executemany(
    "INSERT INTO order_items (order_item_id, order_id, product_id, quantity, line_total) VALUES (?, ?, ?, ?, ?)",
    order_items_data,
)

payment_methods = ["Credit Card", "PayPal", "Wire Transfer", "Gift Card"]
payment_statuses = ["Completed", "Pending", "Refunded"]
payments_data = []
for idx, order in enumerate(orders_data, start=1):
    order_date = datetime.strptime(order[2], "%Y-%m-%d")
    payment_date = (order_date + timedelta(days=random.randint(0, 3))).strftime("%Y-%m-%d")
    payments_data.append(
        (
            idx,
            order[0],
            random.choice(payment_methods),
            payment_date,
            order[3],
            random.choice(payment_statuses),
        )
    )

cursor.executemany(
    "INSERT INTO payments (payment_id, order_id, payment_method, payment_date, payment_amount, payment_status) VALUES (?, ?, ?, ?, ?, ?)",
    payments_data,
)

issue_types = [
    "Shipping Delay", "Damaged Item", "Billing Question", "Technical Issue", "Account Access", "Feature Request",
]
priorities = ["Low", "Medium", "High"]

tickets_data = []
base_ticket_date = datetime(2024, 5, 1)
for ticket_id in range(1, 16):
    user_id = random.choice(users_data)[0]
    created_at = (base_ticket_date - timedelta(days=random.randint(0, 60))).strftime("%Y-%m-%d")
    resolved = random.choice([True, False])
    if resolved:
        resolved_at = (datetime.strptime(created_at, "%Y-%m-%d") + timedelta(days=random.randint(1, 10))).strftime("%Y-%m-%d")
        status = random.choice(["Resolved", "Closed"])
    else:
        resolved_at = None
        status = random.choice(["Open", "In Progress"])
    tickets_data.append((ticket_id, user_id, random.choice(issue_types), random.choice(priorities), status, created_at, resolved_at))

cursor.executemany(
    "INSERT INTO support_tickets (ticket_id, user_id, issue_type, priority, status, created_at, resolved_at) VALUES (?, ?, ?, ?, ?, ?, ?)",
    tickets_data,
)

conn.commit()
conn.close()

print("Created Employee_Information.db with refreshed demo tables.")


Created Employee_Information.db with refreshed demo tables.


In [2]:

import sqlite3

DB_PATH = "Employee_Information.db"
tables = [
    "countries",
    "states",
    "users",
    "user_profiles",
    "addresses",
    "products",
    "orders",
    "order_items",
    "payments",
    "support_tickets",
]

with sqlite3.connect(DB_PATH) as conn:
    cursor = conn.cursor()
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"{table:16} -> {count:2d} rows")


countries        -> 15 rows
states           -> 20 rows
users            -> 30 rows
user_profiles    -> 30 rows
addresses        -> 30 rows
products         -> 12 rows
orders           -> 35 rows
order_items      -> 35 rows
payments         -> 35 rows
support_tickets  -> 15 rows


In [3]:
from pathlib import Path
import sqlite3

DB_PATH = "Employee_Information.db"
info_dir = Path("data/data_information")
info_dir.mkdir(parents=True, exist_ok=True)

table_docs = {
    "countries": {
        "description": "Country dimension with ISO codes and macro statistics.",
        "columns": [
            ("country_id", "INTEGER", "Primary key."),
            ("country_name", "TEXT", "Common name of the country."),
            ("iso_code", "TEXT", "Two-letter ISO code used across the model."),
            ("region", "TEXT", "Global region grouping."),
            ("population", "INTEGER", "Latest rounded population estimate."),
        ],
    },
    "states": {
        "description": "First-level administrative divisions for select countries.",
        "columns": [
            ("state_id", "INTEGER", "Primary key."),
            ("state_name", "TEXT", "Name of the state or province."),
            ("country_id", "INTEGER", "Links back to countries.country_id."),
            ("abbreviation", "TEXT", "Postal-style short code."),
            ("state_capital", "TEXT", "Administrative capital city."),
        ],
    },
    "users": {
        "description": "Core user dimension with signup data and geographic tie-in.",
        "columns": [
            ("user_id", "INTEGER", "Primary key."),
            ("first_name", "TEXT", "User first name."),
            ("last_name", "TEXT", "User last name."),
            ("email", "TEXT", "Unique contact email."),
            ("country_id", "INTEGER", "Country of residence."),
            ("signup_date", "TEXT", "Date the account was created."),
        ],
    },
    "user_profiles": {
        "description": "Extended profile attributes that are optional for each user.",
        "columns": [
            ("profile_id", "INTEGER", "Primary key mirrors user_id."),
            ("user_id", "INTEGER", "Foreign key to users."),
            ("birth_date", "TEXT", "ISO-8601 birth date."),
            ("gender", "TEXT", "Self-identified gender."),
            ("language_preference", "TEXT", "Preferred communication language."),
            ("loyalty_tier", "TEXT", "Reward tier assigned by CRM."),
        ],
    },
    "addresses": {
        "description": "Physical mailing addresses captured for logistics workflows.",
        "columns": [
            ("address_id", "INTEGER", "Primary key."),
            ("user_id", "INTEGER", "Owner of the address."),
            ("address_line1", "TEXT", "Street number and name."),
            ("city", "TEXT", "City or locality."),
            ("state", "TEXT", "State/region text."),
            ("postal_code", "TEXT", "Postal or ZIP code."),
            ("country_id", "INTEGER", "Country reference."),
        ],
    },
    "products": {
        "description": "Active product catalog used by the commerce flows.",
        "columns": [
            ("product_id", "INTEGER", "Primary key."),
            ("product_name", "TEXT", "Display name."),
            ("category", "TEXT", "High-level grouping."),
            ("unit_price", "REAL", "Unit selling price."),
            ("availability_status", "TEXT", "Inventory summary."),
        ],
    },
    "orders": {
        "description": "Order fact table capturing monetary totals and order state.",
        "columns": [
            ("order_id", "INTEGER", "Primary key."),
            ("user_id", "INTEGER", "Customer placing the order."),
            ("order_date", "TEXT", "Date submitted."),
            ("total_amount", "REAL", "Order total in transaction currency."),
            ("currency", "TEXT", "Three-letter currency code."),
            ("status", "TEXT", "Fulfillment lifecycle state."),
        ],
    },
    "order_items": {
        "description": "Line-level breakdown of products included on each order.",
        "columns": [
            ("order_item_id", "INTEGER", "Primary key."),
            ("order_id", "INTEGER", "Links to orders."),
            ("product_id", "INTEGER", "Product that was purchased."),
            ("quantity", "INTEGER", "Units purchased."),
            ("line_total", "REAL", "Extended price for the line."),
        ],
    },
    "payments": {
        "description": "Payment attempts/settlements for each order.",
        "columns": [
            ("payment_id", "INTEGER", "Primary key."),
            ("order_id", "INTEGER", "Order covered by the payment."),
            ("payment_method", "TEXT", "Instrument such as credit card or PayPal."),
            ("payment_date", "TEXT", "Posted date."),
            ("payment_amount", "REAL", "Amount collected."),
            ("payment_status", "TEXT", "Clearing state."),
        ],
    },
    "support_tickets": {
        "description": "Support center issues raised by users after purchase.",
        "columns": [
            ("ticket_id", "INTEGER", "Primary key."),
            ("user_id", "INTEGER", "Customer who opened the ticket."),
            ("issue_type", "TEXT", "Categorized topic."),
            ("priority", "TEXT", "Low/Medium/High severity."),
            ("status", "TEXT", "Ticket workflow status."),
            ("created_at", "TEXT", "Date opened."),
            ("resolved_at", "TEXT", "Date resolved if applicable."),
        ],
    },
}

with sqlite3.connect(DB_PATH) as conn:
    cursor = conn.cursor()
    for table, meta in table_docs.items():
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        rows = cursor.fetchone()[0]
        lines = [
            f"Table: {table}",
            f"Description: {meta['description']}",
            f"Rows: {rows}",
            "",
            "Columns:",
        ]
        for column, dtype, detail in meta["columns"]:
            lines.append(f"- {column} ({dtype}) - {detail}")
        target = info_dir / f"{table}.txt"
        target.write_text("\n".join(lines))
        print(f"Documented {table} -> {target}")

Documented countries -> data/data_information/countries.txt
Documented states -> data/data_information/states.txt
Documented users -> data/data_information/users.txt
Documented user_profiles -> data/data_information/user_profiles.txt
Documented addresses -> data/data_information/addresses.txt
Documented products -> data/data_information/products.txt
Documented orders -> data/data_information/orders.txt
Documented order_items -> data/data_information/order_items.txt
Documented payments -> data/data_information/payments.txt
Documented support_tickets -> data/data_information/support_tickets.txt
